In [1]:
!pip install pytorch-lightning --quiet
import pytorch_lightning as pl

     |████████████████████████████████| 849kB 5.9MB/s 
     |████████████████████████████████| 112kB 38.1MB/s 
     |████████████████████████████████| 184kB 25.4MB/s 
     |████████████████████████████████| 829kB 30.8MB/s 
     |████████████████████████████████| 276kB 37.7MB/s 
     |████████████████████████████████| 1.3MB 35.4MB/s 
     |████████████████████████████████| 143kB 40.2MB/s 
     |████████████████████████████████| 296kB 39.2MB/s 


In [11]:
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import MNIST
from torchvision import transforms
import pytorch_lightning as pl
import os

In [18]:
print(torch.__version__)

1.8.1+cu101


In [19]:
class smallAndSmartModel(pl.LightningModule):

      def __init__(self):
        super(smallAndSmartModel, self).__init__()
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1,28,kernel_size=5),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(28,10,kernel_size=2),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2))
        self.dropout1=torch.nn.Dropout(0.25)
        self.fc1=torch.nn.Linear(250,18)
        self.dropout2=torch.nn.Dropout(0.08)
        self.fc2=torch.nn.Linear(18,10)
        
      #This contains the manupulation on data that needs to be done only once such as downloading it
      def prepare_data(self):
          MNIST(os.getcwd(), train=True, download =True)
          MNIST(os.getcwd(), train=False, download =True)
      
      def train_dataloader(self):
          #This is an essential function. Needs to be included in the code
          #See here i have set download to false as it is already downloaded in prepare_data
          mnist_train=MNIST(os.getcwd(), train=True, download =False,transform=transforms.ToTensor())
          
          #Dividing into validation and training set
          self.train_set, self.val_set= random_split(mnist_train,[55000,5000])
          
          return DataLoader(self.train_set,batch_size=128)
          
      def val_dataloader(self):
          # OPTIONAL
          return DataLoader(self.val_set, batch_size=128)

      def test_dataloader(self):
          # OPTIONAL
          return DataLoader(MNIST(os.getcwd(), train=False, download=False, transform=transforms.ToTensor()), batch_size=128)

      def forward(self,x):
          x=self.layer1(x)
          x=self.layer2(x)
          x=self.dropout1(x)
          x=torch.relu(self.fc1(x.view(x.size(0), -1)))
          x=F.leaky_relu(self.dropout2(x))
          
          return F.softmax(self.fc2(x))


      def configure_optimizers(self):
        # Essential fuction
        #we are using Adam optimizer for our model
        return torch.optim.Adam(self.parameters())

      def training_step(self,batch,batch_idx):
          
          #extracting input and output from the batch
          x,labels=batch
          
          #doing a forward pass
          pred=self.forward(x)
          
          #calculating the loss
          loss = F.nll_loss(pred, labels)
          
          #logs
          logs={"train_loss": loss}
          
          output={
              #REQUIRED: It ie required for us to return "loss"
              "loss": loss,
              #optional for logging purposes
              "log": logs
          }
          
          return output

In [21]:
#abstracts the training, val and test loops

#using one gpu given to us by google colab for max 40 epochs
myTrainer=pl.Trainer(gpus=1, max_epochs=10)

model=smallAndSmartModel()
myTrainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you passed in a val_dataloader but have no validation_step. Skipping validation loop
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params
----------------------------------------
0 | layer1   | Sequential | 728   
1 | layer2   | Sequential | 1.1 K 
2 | dropout1 | Dropout    | 0     
3 | fc1      | Linear     | 4.5 K 
4 | dropout2 | Dropout    | 0     
5 | fc2      | Linear     | 190   
----------------------------------------
6.6 K     Trainable params
0         Non-trainable params
6.6 K     Total params
0.026     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.
# log on a step or aggregate epoch metric to the logger and/or progress bar (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


1